## Set up matching DF's

In [284]:
import pandas as pd
# Set pandas to display all columns
pd.set_option('display.max_columns', None)

In [285]:
toto_raw = pd.read_csv('Data/scrapers/Toto/totoAllSports2025-01-26T14:23:09Z.csv', index_col=0).drop_duplicates()
kambi_raw = pd.read_csv('Data/scrapers/unibet/unibetAllSports2025-01-26T14:23:30Z.csv', index_col=0).drop_duplicates()
# Apply the condition and update outcome_label
toto_raw_football = toto_raw[toto_raw['sport'] == 'Voetbal']
kambi_raw_football = kambi_raw[kambi_raw['sport'] == 'FOOTBALL']
# Adjust odds and line
kambi_raw_football['line'] = kambi_raw_football['line'] / 1000
kambi_raw_football['odds'] = kambi_raw_football['odds'] / 1000

# List of women's competitions
women_competitions_toto = [
    'Portugal Campeonato Nacional, Vrouwen', 'Mexico League MX Vrouwen', 'Australië W-League',
    'Scotland Women\'s Premier League', 'Nederland Eredivisie Vrouwen', 'Engeland FA Super League Vrouwen',
    'Spain Primera División Vrouwen'
]

# Create 'sex' column based on the competition
toto_raw_football['sex'] = toto_raw_football['competition'].apply(
    lambda x: 'W' if x in women_competitions_toto else 'M'
)

# List of women's football competitions
women_competitions_kambi = [
    'A-League (D)', 'Premier League Dames', 'Campeonato Nacional Feminino', 'Liga MX Femenil (D)', 
    'Frauen-Bundesliga', 'Super League (D)', 'Primera División (D)'
]

# Create 'sex' column
kambi_raw_football['sex'] = kambi_raw_football['group_name'].apply(
    lambda x: 'W' if x in women_competitions_kambi else 'M'
)

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_2170/1782787632.py:2: DtypeWarning: Columns (23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  kambi_raw = pd.read_csv('Data/scrapers/unibet/unibetAllSports2025-01-26T14:23:30Z.csv', index_col=0).drop_duplicates()
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_2170/1782787632.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_raw_football['line'] = kambi_raw_football['line'] / 1000
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_2170/1782787632.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

Below is a translation of the abbreviations for the column outcome types into their likely meanings in the context of betting odds:

- `--`: Undefined or No Outcome (e.g., no odds available).
- `AG`: Anytime Goal Scorer (player to score at any time in the match).
- `CS`: Correct Score (predict the exact final score of the event).
- `DC`: Double Chance (cover two outcomes, e.g., Home Win or Draw).
- `DN`: Draw No Bet (stake refunded if the match ends in a draw).
- `FS`: First Scorer (player to score the first goal/point in the event).
- `H1`: First Half Result (outcome at the end of the first half).
- `H2`: Second Half Result (outcome in the second half only).
- `HF`: Half-Time/Full-Time (predict the result at both half-time and full-time).
- `HH`: Head-to-Head (comparison between two participants, e.g., players or teams).
- `HL`: Handicap Line (spread betting; adjust the line to level the playing field).
- `LS`: Last Scorer (player to score the last goal/point in the event).
- `MH`: Match Handicap (spread betting for the entire match).
- `MR`: Match Result (predict the overall winner or draw for the match).
- `OE`: Odd/Even (predict whether the total points/goals scored will be odd or even).
- `WH`: Winning Half (which half will have the higher score).
- `WM`: Winning Margin (predict the margin by which a team/player will win).

In [286]:
# filter down to suitable betting opps
kambi_filtered_football = kambi_raw_football[kambi_raw_football['bet_offer_type_english_name'].isin(['Match', 'Odd/Even', 'Player Occurrence Line', 'Asian Over/Under', 'Over/Under', 'Handicap', 'Asian Handicap', 'Yes/No', 'Head to Head'])]
toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['--', 'DN', 'OE', 'HH', 'HL', 'AG'])]

kambi_filtered_football = kambi_filtered_football
toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['DN', 'OE', 'HH', 'HL', 'AG'])]

# Remove duplicates in toto_filtered_football based on specified columns
toto_filtered_football = toto_filtered_football.drop_duplicates(
    subset=['Event Name', 'Market Name', 'Outcome Name']
)
# Remove duplicates in kambi_filtered_football based on specified columns
kambi_filtered_football = kambi_filtered_football.drop_duplicates(
    subset=['event_name', 'outcome_label', 'criterion_label']
)

# Replace 'Sport Club' with 'SC' in the 'event_name' column
kambi_filtered_football['event_name'] = kambi_filtered_football['event_name'].str.replace('Sport Club', 'SC', regex=False)

## Winnaar: Draw no Bet

In [287]:
from rapidfuzz import process, fuzz
import pandas as pd

# Filter kambi_filtered_football based on criterion_label
filtered_kambi_winnaar = kambi_filtered_football[
    kambi_filtered_football['criterion_label'].str.contains('Draw No Bet')
]

# Filter kambi_filtered_football based on criterion_label
filtered_toto_winnaar = toto_filtered_football[
    toto_filtered_football['Market Name'].str.contains('Draw No Bet')
]

# Preprocess strings: Remove extra whitespace and lowercase
def preprocess(text):
    return ' '.join(text.lower().split())

In [288]:
# import unicodedata

# # Replace '-' with a space and remove accents in the 'names' column
# filtered_kambi_winnaar['event_name'] = filtered_kambi_winnaar['event_name'].apply(
#     lambda x: ''.join(
#         char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
#         if not unicodedata.combining(char)
#     )
# )
# # Replace '-' with a space and remove accents in the 'names' column
# filtered_toto_winnaar['Event Name'] = filtered_toto_winnaar['Event Name'].apply(
#     lambda x: ''.join(
#         char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
#         if not unicodedata.combining(char)
#     )
# )

# # Create 'Team1' and 'Team2'
# filtered_kambi_winnaar[['Team1', 'Team2']] = filtered_kambi_winnaar['event_name'].str.split(' vs ', expand=True)
# filtered_toto_winnaar[['Team1', 'Team2']] = filtered_toto_winnaar['Event Name'].str.split(' vs ', expand=True)

# kambi_events = filtered_kambi_winnaar['event_name'].tolist()

# # Function to find the best match for 'Event Name' in toto_filtered_football
# def find_best_match(event_name):
#     # Use a token-based similarity for better handling of substrings
#     result = process.extractOne(event_name, kambi_events, scorer=fuzz.token_set_ratio, score_cutoff=100)
#     if result is None:  # No match found
#         return None
#     match, score, _ = result
#     return match

# # Apply matching function to toto_filtered_football
# filtered_toto_winnaar['matched_event'] = filtered_toto_winnaar['Event Name'].apply(find_best_match)

In [308]:
import unicodedata
from fuzzywuzzy import process, fuzz

# Replace '-' with a space and remove accents in the 'names' column
filtered_kambi_winnaar['event_name'] = filtered_kambi_winnaar['event_name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

filtered_toto_winnaar['Event Name'] = filtered_toto_winnaar['Event Name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Create 'Team1' and 'Team2'
filtered_kambi_winnaar[['Team1', 'Team2']] = filtered_kambi_winnaar['event_name'].str.split(' vs ', expand=True)
filtered_toto_winnaar[['Team1', 'Team2']] = filtered_toto_winnaar['Event Name'].str.split(' vs ', expand=True)

kambi_events = filtered_kambi_winnaar['event_name'].tolist()

# # Function to find the best match based on Team1 and Team2 fuzzy matching
# def find_best_match(event_name):
#     # Extract Team1 and Team2 from the event_name
#     parts = event_name.split(' vs ')
#     if len(parts) == 2:
#         team1 = parts[0].strip()
#         team2 = parts[1].strip()

#         # Perform fuzzy matching for Team1 and Team2
#         def match_teams(team, kambi_team):
#             return fuzz.token_set_ratio(team, kambi_team) >= 80

#         # Check each event_name in kambi_events
#         for kambi_event in kambi_events:
#             kambi_parts = kambi_event.split(' vs ')
#             if len(kambi_parts) == 2:
#                 kambi_team1 = kambi_parts[0].strip()
#                 kambi_team2 = kambi_parts[1].strip()

#                 # Check if both teams match
#                 if match_teams(team1, kambi_team1) and match_teams(team2, kambi_team2):
#                     return kambi_event  # Return the matched event_name

#     return None  # Return None if no match found

from fuzzywuzzy import fuzz

# Function to find the best match based on Team1 and Team2 fuzzy matching
def find_best_match(event_name):
    # Extract Team1 and Team2 from the event_name
    parts = event_name.split(' vs ')
    if len(parts) == 2:
        team1 = parts[0].strip()#.replace('Athletic', '')
        team2 = parts[1].strip()#.replace('Athletic', '')

        # Perform fuzzy matching for Team1 and Team2 with all kambi events
        def match_teams(team, kambi_team):
            return fuzz.token_set_ratio(team, kambi_team)

        # Perform fuzzy matching for both Team1 and Team2
        kambi_matches = filtered_kambi_winnaar[
            (filtered_kambi_winnaar['Team1'].apply(lambda x: match_teams(team1, x) >= 65)) &
            (filtered_kambi_winnaar['Team2'].apply(lambda x: match_teams(team2, x) >= 65))
        ]
        
        # If there is a match, return the matched event_name and fuzzy match score
        if not kambi_matches.empty:
            # Get the best match score
            best_score_team1 = kambi_matches['Team1'].apply(lambda x: match_teams(team1, x)).max()
            best_score_team2 = kambi_matches['Team2'].apply(lambda x: match_teams(team2, x)).max()

            # Calculate the average score (or you can choose to return another metric)
            average_score = (best_score_team1 + best_score_team2) / 2

            # Get the matched event name
            matched_event_name = kambi_matches['event_name'].iloc[0]

            return matched_event_name, average_score  # Return the matched event and score

    return None, None  # Return None if no match found

# Apply matching function to toto_filtered_football
filtered_toto_winnaar[['matched_event', 'fuzzy_score']] = filtered_toto_winnaar['Event Name'].apply(
    lambda x: pd.Series(find_best_match(x))
)

# # Apply matching function to filtered_toto_winnaar
# filtered_toto_winnaar['matched_event'] = filtered_toto_winnaar['Event Name'].apply(find_best_match)

# Check the results
filtered_toto_winnaar[['Event Name', 'matched_event']].head()

,Event Name,matched_event
0,TSG Hoffenheim vs Eintracht Frankfurt,TSG Hoffenheim vs Eintracht Frankfurt
1,TSG Hoffenheim vs Eintracht Frankfurt,TSG Hoffenheim vs Eintracht Frankfurt
373,Lamia 1964 vs Aris Thessaloniki,Lamia 1964 vs Aris Thessaloniki
374,Lamia 1964 vs Aris Thessaloniki,Lamia 1964 vs Aris Thessaloniki
639,Real Sociedad vs Getafe CF,Real Sociedad vs Getafe


In [311]:
# Get unique records from 'Event Name' and 'matched_event'
matched_events = filtered_toto_winnaar[['Event Name', 'matched_event', 'fuzzy_score']].drop_duplicates()

# Display the result
matched_events[~matched_events['matched_event'].isna()].sort_values(by='fuzzy_score', ascending=False).to_csv('test_match.csv')

In [291]:
# Define a transformation function for standardization
def standardize_draw_no_bet(value):
    if '1e Helft' in value:
        return 'Draw No Bet - 1e Helft'
    elif '2e Helft' in value:
        return 'Draw No Bet - 2e Helft'
    elif 'Draw No Bet' in value:
        return 'Draw No Bet'
    else:
        return value  # Keep original if no match

# Apply the transformation to the relevant columns
filtered_kambi_winnaar['standardized_label'] = filtered_kambi_winnaar['criterion_label'].apply(standardize_draw_no_bet)
filtered_toto_winnaar['standardized_label'] = filtered_toto_winnaar['Market Name'].apply(standardize_draw_no_bet)

# Merge the DataFrames using the standardized column
merged_winnaar = pd.merge(
    filtered_toto_winnaar,
    filtered_kambi_winnaar,
    left_on=['matched_event', 'standardized_label'], 
    right_on=['event_name', 'standardized_label'], 
    how='inner'  # Perform an inner join
)

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_2170/3072039294.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_kambi_winnaar['standardized_label'] = filtered_kambi_winnaar['criterion_label'].apply(standardize_draw_no_bet)
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_2170/3072039294.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_toto_winnaar['standardized_label'] = filtered_toto_winnaar['Market Name'].apply(standardize_draw_no_bet)


In [292]:
# # Filter rows in toto_filtered_football where Market Name == 'Winnaar'
# toto_filtered_football = toto_filtered_football[toto_filtered_football['Market Name'].str.contains('Draw No Bet')]

# Perform a left join on the matched_event_name (from toto) and event_name (from kambi)
merged_df_winnaar = filtered_toto_winnaar.merge(
    filtered_kambi_winnaar,
    left_on=['matched_event', 'standardized_label', 'sex'], 
    right_on=['event_name', 'standardized_label', 'sex'], 
    how='left'
)

merged_df_winnaar = merged_df_winnaar[(merged_df_winnaar['matched_event'].notnull())&(merged_df_winnaar['Outcome SubType'] != merged_df_winnaar['outcome_label'])]

## Over/Under

In [293]:
# filter down to suitable betting opps
kambi_filtered_football = kambi_raw_football[kambi_raw_football['bet_offer_type_english_name'].isin(['Match', 'Odd/Even', 'Player Occurrence Line', 'Asian Over/Under', 'Over/Under', 'Handicap', 'Asian Handicap', 'Yes/No', 'Head to Head'])]
toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['--', 'DN', 'OE', 'HH', 'HL', 'AG'])]

kambi_filtered_football = kambi_raw_football
toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['DN', 'OE', 'HH', 'HL', 'AG'])]

# Remove duplicates in toto_filtered_basketbal based on specified columns
toto_filtered_football = toto_filtered_football.drop_duplicates(
    subset=['Event Name', 'Market Name', 'Outcome Name']
)
# Remove duplicates in kambi_filtered_basketbal based on specified columns
kambi_filtered_football = kambi_filtered_football.drop_duplicates(
    subset=['event_name', 'outcome_label', 'criterion_label']
)

In [294]:
kambi_filtered_football_overunder = kambi_filtered_football[kambi_filtered_football['bet_offer_type_name'].str.contains('Over')]
toto_filtered_football_overunder = toto_filtered_football[(toto_filtered_football['Outcome Name'].str.contains('Over')) | (toto_filtered_football['Outcome Name'].str.contains('Under'))]

In [295]:
kambi_filtered_football_overunder.event_name.unique()

array(['Estoril vs Vitoria Guimarães', 'FC Porto vs Santa Clara',
       'S.C. Braga vs Boavista', 'AVS Futebol Sad vs Gil Vicente',
       'Richards Bay FC vs Orlando Pirates',
       'Polokwane City FC vs The Bees FC',
       'Dunfermline Athletic vs Raith Rovers',
       'Hamilton Academical vs Ayr United',
       'Partick Thistle vs Greenock Morton',
       "Airdrieonians vs Queen's Park", 'Spartans (W) vs Rangers (W)',
       'Aberdeen (W) vs Queens Park (W)', 'Dundee FC vs Hearts',
       'Hibernian vs Aberdeen', 'St Mirren vs St Johnstone',
       'Kilmarnock vs Dundee United', 'Motherwell vs Celtic',
       'Rangers FC vs Ross County', 'SF Damaiense (W) vs S.C. Braga (W)',
       'Mafra vs UD Oliveirense', 'União Leiria vs Portimonense',
       'FC Porto B vs CD de Tondela', 'Torreense vs Benfica B',
       'Sanjoanense vs S.C. Braga B', 'Fafe vs Anadia FC',
       'São João Ver vs Varzim', 'Trofense vs Lusitania Lourosa',
       'Vilaverdense vs Amarante', 'PSV Eindhoven vs Li

In [ ]:
import unicodedata

# Replace '-' with a space and remove accents in the 'names' column
kambi_filtered_football_overunder['event_name'] = kambi_filtered_football_overunder['event_name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Replace '-' with a space and remove accents in the 'names' column
kambi_filtered_football_overunder['criterion_label'] = kambi_filtered_football_overunder['criterion_label'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Replace '-' with a space and remove accents in the 'names' column
kambi_filtered_football_overunder['criterion_english_label'] = kambi_filtered_football_overunder['criterion_english_label'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Replace '-' with a space and remove accents in the 'names' column
toto_filtered_football_overunder['Event Name'] = toto_filtered_football_overunder['Event Name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Replace '-' with a space and remove accents in the 'names' column
toto_filtered_football_overunder['Market Name'] = toto_filtered_football_overunder['Market Name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

kambi_events = kambi_filtered_football_overunder['event_name'].tolist()

# # Function to find the best match for 'Event Name' in toto_filtered_football
# def find_best_match(event_name):
#     # Use a token-based similarity for better handling of substrings
#     result = process.extractOne(event_name, kambi_events, scorer=fuzz.token_set_ratio, score_cutoff=100)
#     if result is None:  # No match found
#         return None
#     match, score = result
#     return match


# # Apply matching function to toto_filtered_football
# toto_filtered_football_overunder['matched_event'] = toto_filtered_football_overunder['Event Name'].apply(find_best_match)
# Perform a left join to bring in 'matched_event' and 'fuzzy_score' from matched_events
toto_filtered_football_overunder = toto_filtered_football_overunder.merge(
    matched_events,
    on='Event Name',  # Join on the 'Event Name' column
    how='left'  # Ensure all rows in toto_filtered_football_overunder are retained
)

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_2170/2219985270.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_overunder['event_name'] = kambi_filtered_football_overunder['event_name'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_2170/2219985270.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_overunder['criterion_label'] = kambi_filtered_football_overunder['criterion_label'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhn

In [297]:
# Get unique records from 'Event Name' and 'matched_event'
unique_records = toto_filtered_football_overunder[['Event Name', 'matched_event']].drop_duplicates()

# Display the result
unique_records[~unique_records['matched_event'].isna()]

,Event Name,matched_event
0,TSG Hoffenheim vs Eintracht Frankfurt,TSG Hoffenheim vs Eintracht Frankfurt
12,Lamia 1964 vs Aris Thessaloniki,Lamia 1964 vs Aris Thessaloniki
18,Real Sociedad vs Getafe CF,Real Sociedad vs Getafe
30,Albacete vs Almeria,Albacete Balompie vs Almeria
42,Real Oviedo vs CD Castellon,Real Oviedo vs CD Castellon
...,...,...
5625,SS Monopoli 1966 vs Cavese 1919,Monopoli vs Cavese
5637,SS Turris Calcio vs Sorrento Calcio 1945,Turris vs ASD Sorrento 1945
5649,US Albinoleffe vs AS Giana Erminio,Albinoleffe vs Giana Erminio
5661,US Triestina Calcio 1918 vs Calcio Lecco 1912,Triestina vs Lecco


In [298]:
# toto_filtered_football_overunder.to_csv('Data/Testing/toto_filtered_football_overunder.csv')
# kambi_filtered_football_overunder.to_csv('Data/Testing/kambi_filtered_football_overunder.csv')  

In [299]:
# Create 'OverUnderType'
kambi_filtered_football_overunder['OverUnderType'] = kambi_filtered_football_overunder['criterion_label'].apply(
    lambda x: 'Goals' if 'Doelpunten' in x else 'Other'
)

# Create 'OverUnderTime'
def determine_over_under_time(label):
    if '1e helft' in label.lower():
        return '1e Helft'
    elif '2e helft' in label.lower():
        return '2e Helft'
    elif label[-2:].isdigit():  # Check if the last two characters in the label are digits
        return label.split(' - ')[-1]
    else:
        return 'Full Time'

kambi_filtered_football_overunder['OverUnderTime'] = kambi_filtered_football_overunder['criterion_label'].apply(determine_over_under_time)

# Create 'Team1' and 'Team2'
kambi_filtered_football_overunder[['Team1', 'Team2']] = kambi_filtered_football_overunder['event_name'].str.split(' vs ', expand=True)

# Create 'OverUnderType2'
kambi_filtered_football_overunder['OverUnderType2'] = kambi_filtered_football_overunder.apply(
    lambda row: '1' if row['Team1'] in row['criterion_english_label'] else (
        '2' if row['Team2'] in row['criterion_english_label'] else 'Total team 1 and team 2'
    ),
    axis=1
)

# # Create 'line'
# kambi_filtered_football_overunder['line'] = kambi_filtered_football_overunder['line'] / 1000

# # Creat odds
# kambi_filtered_football_overunder['odds'] = kambi_filtered_football_overunder['odds'] / 1000

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_2170/3155310470.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_overunder['OverUnderType'] = kambi_filtered_football_overunder['criterion_label'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_2170/3155310470.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_overunder['OverUnderTime'] = kambi_filtered_football_overunder['criterion_label'].apply(determine_over_under_time)
/var/

In [300]:
# Create 'OverUnderType'
toto_filtered_football_overunder['OverUnderType'] = toto_filtered_football_overunder['Market Name'].apply(
    lambda x: 'Goals' if 'Goals' in x else 'Other'
)

# Create 'OverUnderTime'
toto_filtered_football_overunder['OverUnderTime'] = toto_filtered_football_overunder['Market Name'].apply(
    lambda x: '1e Helft' if '1e helft' in x.lower() else ('2e Helft' if '2e helft' in x.lower() else 'Full Time')
)

# Create 'Team1' and 'Team2'
toto_filtered_football_overunder[['Team1', 'Team2']] = toto_filtered_football_overunder['Event Name'].str.split(' vs ', expand=True)

# Create 'OverUnderType2'
toto_filtered_football_overunder['OverUnderType2'] = toto_filtered_football_overunder.apply(
    lambda row: '1' if row['Team1'] in row['Market Name'] else (
        '2' if row['Team2'] in row['Market Name'] else 'Total team 1 and team 2'
    ),
    axis=1
)

# Create 'line'
toto_filtered_football_overunder['line'] = toto_filtered_football_overunder['Market Name'].apply(
    lambda x: float(x.split('Over/Under ')[-1]) if 'Over/Under ' in x else None
)

In [301]:
# Perform the merge
merged_football_overunder = pd.merge(
    toto_filtered_football_overunder,
    kambi_filtered_football_overunder,
    left_on=['line', 'OverUnderType', 'OverUnderTime', 'OverUnderType2', 'matched_event', 'sex'],
    right_on=['line', 'OverUnderType', 'OverUnderTime', 'OverUnderType2', 'event_name', 'sex'],
    how='inner'  # Use 'inner' join to get only matching rows; adjust as needed
)
# Keep records with opposite outcomes
merged_football_overunder = merged_football_overunder[merged_football_overunder['outcome_english_label'] != merged_football_overunder['Outcome Name']]

# Display the merged DataFrame
merged_football_overunder.head()

,event_id_x,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport_x,competition,sex,matched_event,fuzzy_score,OverUnderType,OverUnderTime,Team1_x,Team2_x,OverUnderType2,line,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id_y,outcome_id,outcome_label,outcome_english_label,odds,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport_y,group_name,Team1_y,Team2_y
0,6588685,TSG Hoffenheim vs Eintracht Frankfurt,Aantal Goals Over/Under 2.5,Under,2.65,33,20,HL,L,Voetbal,Duitsland Bundesliga,M,TSG Hoffenheim vs Eintracht Frankfurt,100.0,Goals,Full Time,TSG Hoffenheim,Eintracht Frankfurt,Total team 1 and team 2,2.5,2487156395,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021150826,3618414767,Meer dan,Over,1.42,NaN,OT_OVER,2025-01-26T14:13:23Z,21/50,-240.0,OPEN,ENABLED,NaN,NaN,TSG Hoffenheim vs Eintracht Frankfurt,FOOTBALL,NaN,TSG Hoffenheim,Eintracht Frankfurt
3,6588685,TSG Hoffenheim vs Eintracht Frankfurt,Aantal Goals Over/Under 2.5,Over,1.47,47,100,HL,1,Voetbal,Duitsland Bundesliga,M,TSG Hoffenheim vs Eintracht Frankfurt,100.0,Goals,Full Time,TSG Hoffenheim,Eintracht Frankfurt,Total team 1 and team 2,2.5,2487156395,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021150826,3618414775,Minder dan,Under,2.75,NaN,OT_UNDER,2025-01-26T14:13:23Z,7/4,175.0,OPEN,ENABLED,NaN,NaN,TSG Hoffenheim vs Eintracht Frankfurt,FOOTBALL,NaN,TSG Hoffenheim,Eintracht Frankfurt
5,5881322,Real Sociedad vs Getafe CF,Aantal Goals Over/Under 4.5,Over,12.50,23,2,HL,1,Voetbal,Spanje LaLiga,M,Real Sociedad vs Getafe,100.0,Goals,Full Time,Real Sociedad,Getafe CF,Total team 1 and team 2,4.5,2485877312,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021044022,3614023584,Minder dan,Under,NaN,NaN,OT_UNDER,2025-01-26T14:15:12Z,NaN,NaN,SUSPENDED,ENABLED,NaN,NaN,Real Sociedad vs Getafe,FOOTBALL,La Liga,Real Sociedad,Getafe
6,5881322,Real Sociedad vs Getafe CF,Aantal Goals Over/Under 4.5,Under,1.02,1,50,HL,L,Voetbal,Spanje LaLiga,M,Real Sociedad vs Getafe,100.0,Goals,Full Time,Real Sociedad,Getafe CF,Total team 1 and team 2,4.5,2485877312,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021044022,3614023582,Meer dan,Over,23.00,NaN,OT_OVER,2025-01-26T14:15:12Z,22/1,2200.0,OPEN,ENABLED,NaN,NaN,Real Sociedad vs Getafe,FOOTBALL,La Liga,Real Sociedad,Getafe
8,6622938,Albacete vs Almeria,Aantal Goals Over/Under 0.5,Under,8.50,15,2,HL,L,Voetbal,Spanje Segunda Division,M,Albacete Balompie vs Almeria,100.0,Goals,Full Time,Albacete,Almeria,Total team 1 and team 2,0.5,2487174903,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021099840,3618465551,Meer dan,Over,1.01,NaN,OT_OVER,2025-01-26T14:11:50Z,1/100,-10000.0,OPEN,ENABLED,NaN,NaN,Albacete Balompie vs Almeria,FOOTBALL,La Liga 2,Albacete Balompie,Almeria


## Match Winnaar & OverUnder and do Arb Analysis

In [302]:
# Perform the stacked union
total_football = pd.concat([merged_football_overunder, merged_df_winnaar], ignore_index=True, sort=True)
total_football

,Event Name,Market Name,Odds (Decimal),Outcome Name,Outcome SubType,Outcome Type,OverUnderTime,OverUnderType,OverUnderType2,Price Denominator,Price Numerator,Team1_x,Team1_y,Team2_x,Team2_y,away_score,bet_offer_id,bet_offer_type_english_name,bet_offer_type_id,bet_offer_type_name,cash_out_status,changed_date,competition,criterion_english_label,criterion_id,criterion_label,event_id_x,event_id_y,event_name,fuzzy_score,group_name,home_score,lifetime,line,matched_event,occurrence_type,odds,odds_american,odds_fractional,outcome_english_label,outcome_id,outcome_label,participant,sex,sport_x,sport_y,standardized_label,status,type
0,TSG Hoffenheim vs Eintracht Frankfurt,Aantal Goals Over/Under 2.5,2.65,Under,L,HL,Full Time,Goals,Total team 1 and team 2,20,33,TSG Hoffenheim,TSG Hoffenheim,Eintracht Frankfurt,Eintracht Frankfurt,NaN,2.487156e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-01-26T14:13:23Z,Duitsland Bundesliga,Total Goals,1.001160e+09,Totaal Aantal Doelpunten,6588685,1.021151e+09,TSG Hoffenheim vs Eintracht Frankfurt,100.0,NaN,NaN,FULL_TIME,2.5,TSG Hoffenheim vs Eintracht Frankfurt,GOALS,1.42,-240.0,21/50,Over,3.618415e+09,Meer dan,NaN,M,Voetbal,FOOTBALL,NaN,OPEN,OT_OVER
1,TSG Hoffenheim vs Eintracht Frankfurt,Aantal Goals Over/Under 2.5,1.47,Over,1,HL,Full Time,Goals,Total team 1 and team 2,100,47,TSG Hoffenheim,TSG Hoffenheim,Eintracht Frankfurt,Eintracht Frankfurt,NaN,2.487156e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-01-26T14:13:23Z,Duitsland Bundesliga,Total Goals,1.001160e+09,Totaal Aantal Doelpunten,6588685,1.021151e+09,TSG Hoffenheim vs Eintracht Frankfurt,100.0,NaN,NaN,FULL_TIME,2.5,TSG Hoffenheim vs Eintracht Frankfurt,GOALS,2.75,175.0,7/4,Under,3.618415e+09,Minder dan,NaN,M,Voetbal,FOOTBALL,NaN,OPEN,OT_UNDER
2,Real Sociedad vs Getafe CF,Aantal Goals Over/Under 4.5,12.50,Over,1,HL,Full Time,Goals,Total team 1 and team 2,2,23,Real Sociedad,Real Sociedad,Getafe CF,Getafe,NaN,2.485877e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-01-26T14:15:12Z,Spanje LaLiga,Total Goals,1.001160e+09,Totaal Aantal Doelpunten,5881322,1.021044e+09,Real Sociedad vs Getafe,100.0,La Liga,NaN,FULL_TIME,4.5,Real Sociedad vs Getafe,GOALS,NaN,NaN,NaN,Under,3.614024e+09,Minder dan,NaN,M,Voetbal,FOOTBALL,NaN,SUSPENDED,OT_UNDER
3,Real Sociedad vs Getafe CF,Aantal Goals Over/Under 4.5,1.02,Under,L,HL,Full Time,Goals,Total team 1 and team 2,50,1,Real Sociedad,Real Sociedad,Getafe CF,Getafe,NaN,2.485877e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-01-26T14:15:12Z,Spanje LaLiga,Total Goals,1.001160e+09,Totaal Aantal Doelpunten,5881322,1.021044e+09,Real Sociedad vs Getafe,100.0,La Liga,NaN,FULL_TIME,4.5,Real Sociedad vs Getafe,GOALS,23.00,2200.0,22/1,Over,3.614024e+09,Meer dan,NaN,M,Voetbal,FOOTBALL,NaN,OPEN,OT_OVER
4,Albacete vs Almeria,Aantal Goals Over/Under 0.5,8.50,Under,L,HL,Full Time,Goals,Total team 1 and team 2,2,15,Albacete,Albacete Balompie,Almeria,Almeria,NaN,2.487175e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-01-26T14:11:50Z,Spanje Segunda Division,Total Goals,1.001160e+09,Totaal Aantal Doelpunten,6622938,1.021100e+09,Albacete Balompie vs Almeria,100.0,La Liga 2,NaN,FULL_TIME,0.5,Albacete Balompie vs Almeria,GOALS,1.01,-10000.0,1/100,Over,3.618466e+09,Meer dan,NaN,M,Voetbal,FOOTBALL,NaN,OPEN,OT_OVER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,US Albinoleffe vs AS Giana Erminio,Draw No Bet,2.35,AS Giana Erminio,2,DN,NaN,NaN,NaN,20,27,US Albinoleffe,Albinoleffe,AS Giana Erminio,Giana Erminio,NaN,2.488813e+09,Match,2.0,Wedstrijd,DISABLED,2025-01-26T10:29:16Z,Italy Serie C,Draw No Bet,1.001160e+09,Draw No Bet,6476129,1.021227e+09,Albinoleffe vs Giana Erminio,100.0,Serie C Girone A,NaN,FULL_TIME,NaN,Albinoleffe vs Giana Erminio,GOALS,1.45,-225.0,4/9,1,3.624083e+09,1,Albinoleffe,M,Voetbal,FOOTBALL,Draw No Bet,OPEN,1
1002,US Triestina Calcio 1918 vs Calcio Lecco 1912,Draw No Bet,1.29,US Triesti

In [303]:
# Calculate Arbitrage Percentage
total_football['Arbitrage Percentage'] = (1 / total_football['Odds (Decimal)'] + 1 / total_football['odds']) * 100

# Identify Arbitrage Opportunities
total_football['Is Arbitrage'] = total_football['Arbitrage Percentage'] < 100

# Calculate Optimal Stakes if Arbitrage
total_stake = 1000
total_football['Stake A'] = total_football.apply(
    lambda row: (1 / row['Odds (Decimal)'] / (1 / row['Odds (Decimal)'] + 1 / row['odds']) * total_stake) if row['Is Arbitrage'] else 0,
    axis=1
)
total_football['Stake B'] = total_football.apply(
    lambda row: (1 / row['odds'] / (1 / row['Odds (Decimal)'] + 1 / row['odds']) * total_stake) if row['Is Arbitrage'] else 0,
    axis=1
)

total_football[['Event Name', 'Market Name', 'Outcome Name', 'outcome_label', 'Odds (Decimal)', 'odds', 'Arbitrage Percentage', 'Is Arbitrage', 'Stake A', 'Stake B']]    

,Event Name,Market Name,Outcome Name,outcome_label,Odds (Decimal),odds,Arbitrage Percentage,Is Arbitrage,Stake A,Stake B
0,TSG Hoffenheim vs Eintracht Frankfurt,Aantal Goals Over/Under 2.5,Under,Meer dan,2.65,1.42,108.158384,False,0,0
1,TSG Hoffenheim vs Eintracht Frankfurt,Aantal Goals Over/Under 2.5,Over,Minder dan,1.47,2.75,104.390847,False,0,0
2,Real Sociedad vs Getafe CF,Aantal Goals Over/Under 4.5,Over,Minder dan,12.50,NaN,NaN,False,0,0
3,Real Sociedad vs Getafe CF,Aantal Goals Over/Under 4.5,Under,Meer dan,1.02,23.00,102.387042,False,0,0
4,Albacete vs Almeria,Aantal Goals Over/Under 0.5,Under,Meer dan,8.50,1.01,110.774607,False,0,0
...,...,...,...,...,...,...,...,...,...,...
1001,US Albinoleffe vs AS Giana Erminio,Draw No Bet,AS Giana Erminio,1,2.35,1.45,111.518709,False,0,0
1002,US Triestina Calcio 1918 vs Calcio Lecco 1912,Draw No Bet,US Triestina Calcio 1918,2,1.29,3.30,107.822410,False,0,0
1003,US Triestina Calcio 1918 vs Calcio Lecco 1912,Draw No Bet,Calcio Lecco 1912,1,2.90,1.28,112.607759,False,0,0
1004,Ternana Calcio vs SSD Vis Pesaro 1898,Draw No Bet,Ternana Calcio,2,1.16,4.50,108.429119,False,0,0


In [304]:
total_football[total_football['Is Arbitrage'] == True][['Event Name', 'Market Name', 'Outcome Name', 'outcome_label', 'Odds (Decimal)', 'odds', 'Arbitrage Percentage', 'Is Arbitrage', 'Stake A', 'Stake B']]  

,Event Name,Market Name,Outcome Name,outcome_label,Odds (Decimal),odds,Arbitrage Percentage,Is Arbitrage,Stake A,Stake B


In [305]:
total_football[(total_football['Is Arbitrage'] == True)]#[['Event Name', 'matched_event', 'Market Name', 'criterion_label', 'Outcome Name', 'outcome_label', 'OverUnderType2',
       # 'Odds (Decimal)', 'odds', 'Price Numerator', 'Price Denominator',
       # 'Outcome Type', 'Outcome SubType', 'sport_x', 'competition',
       # 'OverUnderType', 'OverUnderTime', 'Team1_x', 'Team2_x', 'Team1_y', 'Team2_y', 'Arbitrage Percentage',
       # 'Is Arbitrage', 'Stake A', 'Stake B']]

,Event Name,Market Name,Odds (Decimal),Outcome Name,Outcome SubType,Outcome Type,OverUnderTime,OverUnderType,OverUnderType2,Price Denominator,Price Numerator,Team1_x,Team1_y,Team2_x,Team2_y,away_score,bet_offer_id,bet_offer_type_english_name,bet_offer_type_id,bet_offer_type_name,cash_out_status,changed_date,competition,criterion_english_label,criterion_id,criterion_label,event_id_x,event_id_y,event_name,fuzzy_score,group_name,home_score,lifetime,line,matched_event,occurrence_type,odds,odds_american,odds_fractional,outcome_english_label,outcome_id,outcome_label,participant,sex,sport_x,sport_y,standardized_label,status,type,Arbitrage Percentage,Is Arbitrage,Stake A,Stake B
